In [ ]:
# Exercise 2 Create a Chatbot_Leliopoulos_Panagiotis
## Roadmap
#- Train a chatbot model using pre-trained word embeddings such as word2vec or GloVe.
#- Extend the model with a binary classifier that can predict when a change in topic is occurring during a conversation. 
#- This would allow the chatbot to throttle use of dialog context (prepended conversation history) more intelligently.
#- Implement an online learning mechanism and persistent storage so the bot can update its training dataset and incorporate newly learned facts on the fly.

In [1]:
import tensorflow as tf
import tensorlayer as tl
import numpy as np
from tensorlayer.cost import cross_entropy_seq, cross_entropy_seq_with_mask
from tqdm import tqdm
from sklearn.utils import shuffle
from data.Cornell import data
from tensorlayer.models.seq2seq import Seq2seq
from tensorlayer.models.seq2seq_with_attention import Seq2seqLuongAttention
import os

In [2]:
def initial_setup(data_corpus):
    metadata, idx_q, idx_a = data.load_data(PATH='data/{}/'.format(data_corpus))
    (trainX, trainY), (testX, testY), (validX, validY) = data.split_dataset(idx_q, idx_a)
    trainX = tl.prepro.remove_pad_sequences(trainX.tolist())
    trainY = tl.prepro.remove_pad_sequences(trainY.tolist())
    testX = tl.prepro.remove_pad_sequences(testX.tolist())
    testY = tl.prepro.remove_pad_sequences(testY.tolist())
    validX = tl.prepro.remove_pad_sequences(validX.tolist())
    validY = tl.prepro.remove_pad_sequences(validY.tolist())
    return metadata, trainX, trainY, testX, testY, validX, validY

In [3]:
if __name__ == "__main__":
    data_corpus = "Cornell"

    #data preprocessing
    metadata, trainX, trainY, testX, testY, validX, validY = initial_setup(data_corpus)

    # Parameters
    src_len = len(trainX)
    tgt_len = len(trainY)

    assert src_len == tgt_len

    batch_size = 32
    #batch_size = 4    
    n_step = src_len // batch_size
    src_vocab_size = len(metadata['idx2w']) # 8002 (0~8001)
    emb_dim = 1024
    #emb_dim = 64

    word2idx = metadata['w2idx']   # dict  word 2 index
    idx2word = metadata['idx2w']   # list index 2 word

    unk_id = word2idx['unk']   # 1
    pad_id = word2idx['_']     # 0

    start_id = src_vocab_size  # 8002
    end_id = src_vocab_size + 1  # 8003

    word2idx.update({'start_id': start_id})
    word2idx.update({'end_id': end_id})
    idx2word = idx2word + ['start_id', 'end_id']

    src_vocab_size = tgt_vocab_size = src_vocab_size + 2
    
    # We run the code with 50 epochs.
    num_epochs = 50
    #num_epochs = 1
    vocabulary_size = src_vocab_size
    


    def inference(seed, top_n):
        model_.eval()
        seed_id = [word2idx.get(w, unk_id) for w in seed.split(" ")]
        sentence_id = model_(inputs=[[seed_id]], seq_length=20, start_token=start_id, top_n = top_n)
        sentence = []
        for w_id in sentence_id[0]:
            w = idx2word[w_id]
            if w == 'end_id':
                break
            sentence = sentence + [w]
        return sentence

    decoder_seq_length = 20
    #decoder_seq_length = 5
   
    # class, Seq2Seq that provides high level abstractions for building the graph,
    # training, evaluation, saving and restoring trained model. 
    # We created a class, Seq2Seq that provides high level abstractions for building the graph, 
    # training, evaluation, saving and restoring trained model.
    # The constructor takes the below parameters as input: n_layer=3,n_units=256,num_epochs=50,learning_rate=0.001
    # we create the LSTM cell, the most essential part of the graph
        

    model_ = Seq2seq(
        decoder_seq_length = decoder_seq_length,
        cell_enc=tf.keras.layers.GRUCell,
        cell_dec=tf.keras.layers.GRUCell,
        n_layer=3,
        #n_layer=1,        
        n_units=256,
        #n_units=8,        
        embedding_layer=tl.layers.Embedding(vocabulary_size=vocabulary_size, embedding_size=emb_dim),
        )
    

    # Uncomment below statements if you have already saved the model
    # load_weights = tl.files.load_npz(name='model.npz')
    # tl.files.assign_weights(load_weights, model_)
    
    #We use another high level function sequence_loss, to get the expression for loss. 
    #Then, we build a train operation that minimizes the loss.

    optimizer = tf.optimizers.Adam(learning_rate=0.001)
    model_.train()

    # Questions for chatting with the bot.
    seeds = ["Hi","how are you?","really?","how're you?","where're you from?","which city?","you don't like california?",
             "why not?","how old are you?","what's your name?","what's your name?"]
    
    for epoch in range(num_epochs):
        model_.train()
        trainX, trainY = shuffle(trainX, trainY, random_state=0)
        total_loss, n_iter = 0, 0
        for X, Y in tqdm(tl.iterate.minibatches(inputs=trainX, targets=trainY, batch_size=batch_size, shuffle=False), 
                        total=n_step, desc='Epoch[{}/{}]'.format(epoch + 1, num_epochs), leave=False):

            X = tl.prepro.pad_sequences(X)
            _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=end_id)
            _target_seqs = tl.prepro.pad_sequences(_target_seqs, maxlen=decoder_seq_length)
            _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=start_id, remove_last=False)
            _decode_seqs = tl.prepro.pad_sequences(_decode_seqs, maxlen=decoder_seq_length)
            _target_mask = tl.prepro.sequences_get_mask(_target_seqs)

            with tf.GradientTape() as tape:
                ## compute outputs
                output = model_(inputs = [X, _decode_seqs])
                
                output = tf.reshape(output, [-1, vocabulary_size])
                ## compute loss and update model
                #we build a train operation that minimizes the loss.
                loss = cross_entropy_seq_with_mask(logits=output, target_seqs=_target_seqs, input_mask=_target_mask)

                grad = tape.gradient(loss, model_.all_weights)
                optimizer.apply_gradients(zip(grad, model_.all_weights))
            
            total_loss += loss
            n_iter += 1

        # printing average loss after every epoch
        print('Epoch [{}/{}]: loss {:.4f}'.format(epoch + 1, num_epochs, total_loss / n_iter))

        for seed in seeds:
            print("Query >", seed)
            top_n = 3
            for i in range(top_n):
                sentence = inference(seed, top_n)
                print(" >", ' '.join(sentence))

        tl.files.save_npz(model_.all_weights, name='model.npz')

[TL] Embedding embedding_1: (8004, 1024)
[TL] RNN rnn_1: cell: GRUCell, n_units: 256
[TL] RNN rnn_2: cell: GRUCell, n_units: 256
[TL] RNN rnn_3: cell: GRUCell, n_units: 256
[TL] RNN rnn_4: cell: GRUCell, n_units: 256
[TL] RNN rnn_5: cell: GRUCell, n_units: 256
[TL] RNN rnn_6: cell: GRUCell, n_units: 256
[TL] Reshape reshape_1
[TL] Dense  dense_1: 8004 No Activation
[TL] Reshape reshape_2
[TL] Reshape reshape_3


Epoch [1/50]: loss 5.3819
Query > Hi
 > i dont think you know
 > i know i dont think you can
 > i know
Query > how are you?
 > i know you have to be unk unk and i cant be unk
 > i just think you can do
 > i dont think you were in the same time and
Query > really?
 > i know
 > i know you dont want to be able to do
 > i cant get up with the hospital of the same time
Query > how're you?
 > i dont want you to know that unk
 > i cant do that i know you know that
 > i know you
Query > where're you from?
 > i dont know i know
 > i know
 > i just want to be a unk unk
Query > which city?
 > i just think so i know you were a little time
 > i know
 > i just know i know you
Query > you don't like california?
 > i just want to be a lot of my own
 > i just know
 > i dont know
Query > why not?
 > i know you dont know that i dont want you to
 > i am
 > i am unk
Query > how old are you?
 > i just think so
 > i dont know that i am i know
 > i dont know that i am
Query > what's your name?
 > i didnt know

Epoch [2/50]: loss 4.7175
Query > Hi
 > i think so i was unk
 > i know what you want
 > i dont know
Query > how are you?
 > i was thinking i am
 > i dont know i know
 > i was unk
Query > really?
 > i dont think so i am
 > i know
 > i dont want to know what i was
Query > how're you?
 > i dont want to get a little unk
 > i think you should be able
 > i dont know
Query > where're you from?
 > i dont think you were
 > i know what i was
 > i know
Query > which city?
 > i dont want you to be unk
 > i dont know i dont think i dont know that
 > i think i know that i dont think so
Query > you don't like california?
 > i think you were unk
 > i dont think so i think so i am unk
 > i think you should be unk to the police of the other
Query > why not?
 > i dont want to talk to you i dont know what i want you
 > i dont want to get a drink to the other unk of
 > i dont know
Query > how old are you?
 > i dont want to get it out of here to see you in
 > i dont think i know what to do
 > i dont think s

Epoch [3/50]: loss 4.4590
Query > Hi
 > i dont want you to see him in this place in unk
 > i know what you want is it to be a unk
 > i know what i think i was looking at the same thing i am not
Query > how are you?
 > i dont think so
 > i dont know
 > i dont want to see you i want it
Query > really?
 > i dont know
 > i dont think you know what it takes
 > i know
Query > how're you?
 > unk you are unk unk and a half
 > unk is unk unk
 > unk is unk in a minute
Query > where're you from?
 > i dont know
 > i know i was going to see him
 > i know i dont want to be a little girl
Query > which city?
 > i dont know
 > i dont know
 > i dont know what it is
Query > you don't like california?
 > i think i should
 > i dont think so
 > i think its a unk unk
Query > why not?
 > i know what it is about the unk
 > i think so you should be able a little more unk
 > i think you were unk
Query > how old are you?
 > unk is unk
 > unk the unk
 > unk the unk
Query > what's your name?
 > unk is a good idea t

Epoch [4/50]: loss 4.2459
Query > Hi
 > i didnt know you were in a unk
 > i think i should be surprised that i
 > i dont think so
Query > how are you?
 > unk unk
 > unk unk
 > unk unk and a single one
Query > really?
 > i dont know where it is unk is it unk unk unk is not the problem
 > i dont know where to get
 > i didnt know
Query > how're you?
 > unk is unk for unk with you with the unk
 > unk is unk unk
 > unk unk and i dont know
Query > where're you from?
 > i didnt say it i dont know what to do
 > i think so
 > i think you are unk unk unk
Query > which city?
 > i didnt have any of it to me
 > i dont think so but you should be a unk unk
 > i think so you should get a lot of questions with the people
Query > you don't like california?
 > i dont think i can see
 > i dont want to go home with me i want you out
 > i think i can see
Query > why not?
 > i think its unk
 > i think its true than you
 > i think you should have been a little
Query > how old are you?
 > i know
 > i know i do

...

Epoch [46/50]: loss 1.8538
Query > Hi
 > i dont know why we can talk to her in
 > i cant
 > i cant
Query > how are you?
 > well help mock missus
 > well help mock missus sake
 > well now dont make sure sal are
Query > really?
 > i dont know where tits youre looking
 > i could buy the move today
 > i cant put that out
Query > how're you?
 > unk unk you draw pictures with the others ticket
 > unk unk did she say anything
 > unk unk did you
Query > where're you from?
 > i cant pretend to live with willie
 > i do remember a dollar while i thought i knew something about your fucking car
 > i am the ones
Query > which city?
 > id rather be smiling now especially if you dont come out
 > id draw that from my last idea i was feeling thin stuff for
 > id draw that from my entire design hanging years
Query > you don't like california?
 > whos that from now you boys unk hysterical
 > whos going into a baby and
 > whos that
Query > why not?
 > wade saw him kinds
 > wade you talking to death now
 > 

Epoch [47/50]: loss 1.8561
Query > Hi
 > he was in here
 > he didnt take cocaine
 > he didnt kill her
Query > how are you?
 > well all told you
 > well expecting you into beer
 > well expecting him to make love with things and champagne wheres my own
Query > really?
 > he must like to put it on the river for cocaine
 > he didnt kill her
 > he was
Query > how're you?
 > unk unk how come
 > unk you mean hitler unk you
 > unk unk
Query > where're you from?
 > unk actually
 > unk actually short actually unk
 > unk unk touch
Query > which city?
 > its over
 > its disappeared i got problems
 > its disappeared i got your problems right now i guess she can move right back
Query > you don't like california?
 > im fine
 > im fine
 > im happy
Query > why not?
 > she was in a hero by any time
 > she was in a hero in new years
 > she was into my unk by the way his unk was dead now now
Query > how old are you?
 > six and a unk
 > six and seventy feet you owe the unk
 > six to seven unk
Query > what'

Epoch [48/50]: loss 1.8570
Query > Hi
 > he was with a knife in unk
 > he must come after me see you in
 > he was highly understanding to him now and hes programmed me
Query > how are you?
 > well some are a fine girl
 > well some unk are anywhere
 > well some are most fine things looking forward anywhere
Query > really?
 > he was here
 > he dumped us over some of the people were going to find him out edgar
 > he dumped us here bill
Query > how're you?
 > unk my exact unk
 > unk my unk please than you and
 > unk you say mama
Query > where're you from?
 > yes very for the little help
 > yes very cool i heard
 > yes yes yes yes but im definitely
Query > which city?
 > id better drop out here
 > id rather stand on him
 > id like anyone to see you again soon as you want one thing gentlemen sorry
Query > you don't like california?
 > im fine i just had to wait till her act alone everythings
 > im unk not closed
 > im just going down for a minute wanna help childs pull up real
Query > why no

Epoch [49/50]: loss 1.8615
Query > Hi
 > he left
 > he left
 > he left
Query > how are you?
 > well falling in inside and we left it before sometime they just take the creep to impress the food
 > well call or the name of the name about
 > well falling out of hand and it looks like fbi
Query > really?
 > he dumped us loose at home
 > he dumped us rules in america promised and cops didnt he
 > he sits in a tub political accident it was gone later right
Query > how're you?
 > round in morning i always saw him like the news
 > round a port in their unk as long
 > round in morning
Query > where're you from?
 > sid actually
 > sid actually can i come in
 > sid why are all right youre not getting off the city sailor youre all dead
Query > which city?
 > id have to
 > id say
 > id have paid
Query > you don't like california?
 > whos the next piece i feel
 > whos that
 > whos a unk
Query > why not?
 > it doesnt even feed words
 > it was not a bitch since
 > it doesnt take any help on that
Quer

Epoch [50/50]: loss 1.8682
Query > Hi
 > he must have taken his unk and already his unk
 > he must come back here
 > he broke up and ended
Query > how are you?
 > excuse us wont happen next to madam george
 > excuse me
 > excuse johnnie or next
Query > really?
 > he belongs to his room and
 > he broke my neck
 > he belongs
Query > how're you?
 > unk degrees round spirits
 > unk unk did it come to your nights rest tonight tonight tonight kill us boys
 > unk unk did you lose genesis
Query > where're you from?
 > round beside the unk about two weeks
 > round beside the country you know were gone
 > round my unk or so quick in your wife
Query > which city?
 > id better go
 > id have problems number
 > id better go see you in
Query > you don't like california?
 > jungle unk is not ready to be don cookies on the baby tonight man
 > jungle and unk
 > jungle unk
Query > why not?
 > we have a job for the boss
 > we are not healthy
 > we get jobs with a warehouse setting
Query > how old are you?